In [1]:
import json
import subprocess
def get_video_fps(file_path):
    cmd = [
        'ffprobe', 
        '-v', 'quiet', 
        '-print_format', 'json', 
        '-show_streams', 
        file_path
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    data = json.loads(result.stdout)

    if 'streams' in data and len(data['streams']) > 0:
        fps_info = data['streams'][0].get('r_frame_rate', '0/1')
        numerator, denominator = map(int, fps_info.split('/'))
        fps = numerator / denominator
        return fps
    else:
        raise ValueError("Không thể lấy thông tin FPS từ dữ liệu trả về của ffprobe.")

def get_video_duration(input_file):
    cmd = [
        'ffprobe', 
        '-v', 'quiet', 
        '-print_format', 'json', 
        '-show_format', 
        '-show_streams', 
        input_file
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    try:
        data = json.loads(result.stdout)
        if 'format' in data and 'duration' in data['format']:
            return float(data['format']['duration'])
        else:
            raise ValueError("Không tìm thấy thông tin độ dài video.")
    except json.JSONDecodeError:
        raise ValueError("Lỗi phân tích cú pháp JSON từ ffprobe.")

In [4]:
import subprocess
from tqdm import tqdm
import os
import math

def process_video(input_file: str, video_type: str = 'raw', which_chunk: int = 1, fps = 30):
    assert video_type in ['raw', 'processed']
    # Tên file đầu ra
    root_dir = './data/sample_tv360'
    
    if os.path.exists(root_dir):
        if not os.path.exists(os.path.join(root_dir, 'video_raw')):
            os.makedirs(os.path.join(root_dir, 'video_raw'))
        if not os.path.exists(os.path.join(root_dir, 'audio_raw')):
            os.makedirs(os.path.join(root_dir, 'audio_raw'))
            
    raw_video_dir = os.path.join(root_dir, 'video_raw')
    raw_audio_dir = os.path.join(root_dir, 'audio_raw')
    raw_video = f"{raw_video_dir}/video_{video_type}_{which_chunk}.mp4"
    raw_audio = f"{raw_audio_dir}/audio_{video_type}_{which_chunk}.wav"
    # Xử lý video và audio cùng lúc
    video_cmd = [
        'ffmpeg',
        '-i', input_file,
        '-y', 
        '-an', 
        '-r', f'{fps}',
        '-c:v', 'libopenh264',
        raw_video,
    ]
    audio_cmd = [
        'ffmpeg',
        '-i', input_file,
        '-y',
        '-vn',
        '-acodec', 'pcm_s16le',
        '-ar', '44100',
        '-ac','2',
        raw_audio
    ]
    
    subprocess.run(video_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run(audio_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    print(f"Đã xử lý {input_file}:")
    print(f"  - Video không âm thanh: {raw_video}")
    print(f"  - File âm thanh: {raw_audio}")

# Xử lý tất cả các chunk video
number_of_chunks = math.ceil(get_video_duration("sample_tv360.mp4")/get_video_duration("sample_tv360_highlight.mp4"))
fps = get_video_fps("sample_tv360.mp4")
for i in tqdm(range(1, number_of_chunks +1)):  # Giả sử có 10 chunk từ 1 đến n
    input_file = f'./data/sample_tv360/video_chunk/chunk_{i}.mp4'
    if os.path.exists(input_file):
        print(f"\nĐang xử lý {input_file}...")
        process_video(input_file, video_type =  'raw', which_chunk = i, fps = fps)
    else:
        print(f"Không tìm thấy file {input_file}")

print("\nHoàn thành xử lý tất cả các chunk vào audio và video raw")


  0%|          | 0/9 [00:00<?, ?it/s]


Đang xử lý ./data/sample_tv360/video_chunk/chunk_1.mp4...


 11%|█         | 1/9 [04:19<34:37, 259.74s/it]

Đã xử lý ./data/sample_tv360/video_chunk/chunk_1.mp4:
  - Video không âm thanh: ./data/sample_tv360/video_raw/video_raw_1.mp4
  - File âm thanh: ./data/sample_tv360/audio_raw/audio_raw_1.wav

Đang xử lý ./data/sample_tv360/video_chunk/chunk_2.mp4...


 22%|██▏       | 2/9 [08:58<31:35, 270.77s/it]

...

Đã xử lý ./data/sample_tv360/video_chunk/chunk_7.mp4:
  - Video không âm thanh: ./data/sample_tv360/video_raw/video_raw_7.mp4
  - File âm thanh: ./data/sample_tv360/audio_raw/audio_raw_7.wav

Đang xử lý ./data/sample_tv360/video_chunk/chunk_8.mp4...


 89%|████████▉ | 8/9 [35:30<04:27, 267.99s/it]

Đã xử lý ./data/sample_tv360/video_chunk/chunk_8.mp4:
  - Video không âm thanh: ./data/sample_tv360/video_raw/video_raw_8.mp4
  - File âm thanh: ./data/sample_tv360/audio_raw/audio_raw_8.wav

Đang xử lý ./data/sample_tv360/video_chunk/chunk_9.mp4...


100%|██████████| 9/9 [39:48<00:00, 265.35s/it]

Đã xử lý ./data/sample_tv360/video_chunk/chunk_9.mp4:
  - Video không âm thanh: ./data/sample_tv360/video_raw/video_raw_9.mp4
  - File âm thanh: ./data/sample_tv360/audio_raw/audio_raw_9.wav

Hoàn thành xử lý tất cả các chunk vào audio và video raw
